In [ ]:
import pandas as pd

main_headers = ['sNo','sPort','rxPkts','rxBytes','rxError','txPkts','txBytes','txError','portSpeed','portStat','linkStat'] 
final_header = [] 
for i in range(1,6):
  for k in range(4):
    for j in main_headers:
      final_header.append(str(i)+str(k)+j)
final_header.extend(['d11','d12','d13','d14','d21','d22','d23','d25','d31','d32','d33','d34','d35','d41','d43','d44','d45','d52','d53','d54','d55','y'])
#print(len(final_header))
#print(final_header)


df = pd.read_csv('alldata.csv',names=final_header)
#print(df.shape)
df.head(3)


import numpy as np

### droping unusefull columns
errors_0_cols = []
delay_0_cols = []
stats_cols = []
for i in df.columns:
  if 'Error' in i:
    errors_0_cols.append(i)
  elif len(i) == 3 and 'd' in i and i[1]==i[2]:
    delay_0_cols.append(i)
  elif 'portStat' in i or 'linkStat' in i:
    stats_cols.append(i)
#print(errors_0_cols)
#print(delay_0_cols)
#print(stats_cols)


sum_errors = df[errors_0_cols].values
sum_delays = df[delay_0_cols].values
#print(sum_errors.shape)
#print(sum_errors.sum())
#print(sum_delays.sum())

delay_0_cols.extend(stats_cols)
errors_0_cols.extend(delay_0_cols)
#print(len(errors_0_cols))


df.drop(errors_0_cols,axis=1,inplace=True)
df.shape


X = df.drop('y',axis=1).values
#print(X.shape)
y = df['y'].values
#print(y.shape)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                    stratify=y,
                                                    random_state=0)


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

param_grid = {'kneighborsclassifier__n_neighbors': range(1, 10)}
knn_pipe = make_pipeline(StandardScaler(), KNeighborsClassifier())
grid = GridSearchCV(knn_pipe, param_grid, cv=10)
grid.fit(X_train, y_train)
print(grid.best_params_)
print(grid.score(X_test, y_test))

{'kneighborsclassifier__n_neighbors': 1}
0.918918918918919


In [ ]:
from sklearn.metrics import precision_recall_fscore_support

y_pred = grid.predict(X_test)
print('=========================== KNN =================================')
print(precision_recall_fscore_support(y_test, y_pred, average='macro'))
print(precision_recall_fscore_support(y_test, y_pred, average='micro'))
print(precision_recall_fscore_support(y_test, y_pred, average='weighted'))

=========================== KNN =================================
(0.9230063156957584, 0.9185377136077708, 0.9196748909164789, None)
(0.918918918918919, 0.918918918918919, 0.918918918918919, None)
(0.9229623379392496, 0.918918918918919, 0.919813875968918, None)
